<a href="https://colab.research.google.com/github/IEEE-NITK/Quantum_computing/blob/main/Learning/Sujay/Adjoint_differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Created on 1st February 2022 20:39

@author:Sujay Chuttar

Performing adjoint differentiation in quantum circuits
"""

In [4]:
import pennylane as qml
from pennylane import numpy as np

In [6]:
#device to simulate algorithm on
dev = qml.device('default.qubit', wires = 2)

x = ([0.1, 0.2, 0.3])

@qml.qnode(dev, diff_method = 'adjoint')
def circuit(a):
  qml.RX(a[0], wires = 0)
  qml.CNOT(wires = [0, 1])
  qml.RY(a[1], wires = 1)
  qml.RZ(a[2], wires = 1)
  return qml.expval(qml.PauliX(wires = 1))

In [8]:
n_gates = 4
n_params = 3

ops = [
       qml.RX(x[0], wires = 0),
       qml.CNOT(wires = (0, 1)),
       qml.RY(x[1], wires = 1),
       qml.RZ(x[2], wires = 1)
]
M = qml.PauliX(wires = 1)

In [11]:
state = dev._create_basis_state(0)

for op in ops:
  state = dev._apply_operation(state, op)

print(state)

[[9.82601808e-01-0.14850574j 9.85890302e-02+0.01490027j]
 [7.45635195e-04+0.00493356j 7.43148086e-03-0.04917107j]]


In [12]:
bra = dev._apply_operation(state, M)
ket = state

In [13]:
M_expval = np.vdot(bra, ket)
print("vdot: ", M_expval)
print("Qnode: ", circuit(x))

vdot:  (0.18884787122715618+3.634721684493463e-19j)
Qnode:  0.18884787122715618


In [14]:
bra_n = dev._create_basis_state(0)

for op in ops:
  bra_n = dev._apply_operation(bra_n, op)

bra_n = dev._apply_operation(bra_n, M)
bra_n = dev._apply_operation(bra_n, ops[-1].inv())

ops[-1].inv() #returning the operation to an uninverted state

ket_n = dev._create_basis_state(0)

for op in ops[:-1]:
  ket_n = dev._apply_operation(ket_n, op)

M_expval_n = np.vdot(bra_n, ket_n)
print(M_expval_n)

(0.18884787122715616+1.9739809094676298e-18j)


In [15]:
#version 2 method
bra_n_v2 = dev._apply_operation(state, M)
ket_n_v2 = state

ops[-1].inv()

bra_n_v2 = dev._apply_operation(bra_n_v2, ops[-1])
ket_n_v2 = dev._apply_operation(ket_n_v2, ops[-1])

M_expval_n_v2 = np.vdot(bra_n_v2, ket_n_v2)
print(M_expval_n_v2)

(0.18884787122715613+2.9931365520227565e-18j)
